# `CLOUD.jl` - 2D linear advection example

In [2]:
using CLOUD, OrdinaryDiffEq

┌ Info: Precompiling CLOUD [fb992021-99c7-4c2d-a14b-5e48ac4045b2]
└ @ Base loading.jl:1423


## Define the physical problem

In [3]:
θ = π/4 # wave angle
a = sqrt(2) # wave speed
A = 1.0  # amplitude
L = 1.0  # domain length
k = (2*π/L, 2*π/L)  # wave number
T = 0.1/(a*max(abs(cos(θ)),abs(sin(θ)))) # end time for one period

conservation_law = LinearAdvectionEquation((a*cos(θ),a*sin(θ)))
initial_data = InitialDataSine(A,k)
exact_solution = ExactSolution(conservation_law,initial_data);

## Set discretization parameters

In [4]:
M = 4
p = 4
p_map = 4
form = WeakConservationForm(mapping_form=SkewSymmetricMapping(), 
    inviscid_numerical_flux=LaxFriedrichsNumericalFlux(0.0))
strategy = ReferenceOperator()

ode_algorithm = CarpenterKennedy2N54()
dt = 0.01*(L/M)/(a*(2*p+1));
write_interval = floor(Int, T/(dt*10));

## Set up a `DGSEM` scheme with LG quadrature on quadrilaterals of degree `p` with `M` edges in each direction

In [5]:
#reference_approximation = ReferenceApproximation(
#    DGSEM(p), Quad(), mapping_degree=p_map, N_plot=5)

reference_approximation = ReferenceApproximation(
    DGMulti(p), Tri(), mapping_degree=p_map, N_plot=2)

mesh = warp_mesh(uniform_periodic_mesh(
    reference_approximation.reference_element, 
    ((0.0,L),(0.0,L)), (M,M)), 
    reference_approximation.reference_element, 0.2)

spatial_discretization = SpatialDiscretization(mesh, 
    reference_approximation)

results_path = save_project(conservation_law,
     spatial_discretization, initial_data, form, (0.0, T), strategy,
     "results/advection_2d_dgsem/", overwrite=true, clear=true)

"results/advection_2d_dgsem/"

## Run the solver

In [6]:
ode_problem_1 = semidiscretize(conservation_law,
    spatial_discretization,
    initial_data, 
    form,
    (0.0, T),
    strategy);

save_solution(ode_problem_1.u0, 0.0, results_path, 0)

CLOUD_reset_timer!()
sol = solve(ode_problem_1, ode_algorithm, adaptive=false, dt=dt,
    save_everystep=false, callback=save_callback(results_path, write_interval))
CLOUD_print_timer()

save_solution(last(sol.u), last(sol.t), results_path, "final")

 ────────────────────────────────────────────────────────────────────────────────
            Thread 1                    Time                    Allocations      
                               ───────────────────────   ────────────────────────
       Tot / % measured:            8.04s /  33.1%           3.39GiB /  55.0%    

 Section               ncalls     time    %tot     avg     alloc    %tot      avg
 ────────────────────────────────────────────────────────────────────────────────
 local residual         81.6k    2.45s   92.3%  30.1μs   1.79GiB   96.1%  23.0KiB
   apply operators      81.6k    1.39s   52.3%  17.0μs   0.99GiB   53.0%  12.7KiB
     volume terms       81.6k    667ms   25.1%  8.17μs    575MiB   30.1%  7.21KiB
     mass matrix solve  81.6k    194ms    7.3%  2.38μs    186MiB    9.7%  2.33KiB
     facet terms        81.6k    112ms    4.2%  1.37μs   94.2MiB    4.9%  1.18KiB
     mul test function  81.6k   58.2ms    2.2%   713ns   33.9MiB    1.8%     436B
   gather ext s

## Calculate the integral L<sup>2</sup> error norm and conservation/energy error

In [7]:
error_analysis = ErrorAnalysis(results_path, conservation_law, 
    spatial_discretization)
conservation_analysis = PrimaryConservationAnalysis(results_path, 
    conservation_law, spatial_discretization)
energy_analysis = EnergyConservationAnalysis(results_path, 
    conservation_law, spatial_discretization)

println("L2 error:")
println(analyze(error_analysis, last(sol.u), initial_data, T)...)
println("Conservation (initial/final/diff):")
println(analyze(conservation_analysis)...)
println("Energy (initial/final/diff):")
println(analyze(energy_analysis)...)

L2 error:
0.41575516319051786
Conservation (initial/final/diff):
[-1.8792023555769862e-5][-1.8792023555948972e-5][-1.7911019889460533e-16]
Energy (initial/final/diff):
[0.12490255166391541][0.1249025516639116][-3.802513859341161e-15]


## Write output to VTK file

In [8]:
postprocess_vtk(spatial_discretization, string(results_path,"solution.vtu"), last(sol.u));